# 🎮 Play Utala: KAOS 9 vs AI

Interactive gameplay - test your skills against AI opponents!

---

## Setup

Run this cell first to install the game:

In [ ]:
import micropip
import js

# Detect if we're on localhost or GitHub Pages
origin = str(js.location.origin)
pathname = str(js.location.pathname)

if "localhost" in origin or "127.0.0.1" in origin:
    # Local development
    wheel_url = f'{origin}/pyodide/utala_kaos_9-0.1.1-py3-none-any.whl'
else:
    # GitHub Pages (has /games/ in path)
    wheel_url = f'{origin}/games/pyodide/utala_kaos_9-0.1.1-py3-none-any.whl'

print(f"Installing from: {wheel_url}")
await micropip.install(wheel_url)
print("✓ Game installed successfully!")

## Import Game Components

In [ ]:
from utala.agents.random_agent import RandomAgent
from utala.agents.heuristic_agent import HeuristicAgent
from utala.agents.monte_carlo_agent import FastMonteCarloAgent
from utala.agents.human_agent import HumanAgent
from utala.engine import GameEngine
from utala.state import Player, Phase
from utala.actions import get_action_space

print("✓ Imports complete!")

## Select Your Opponent

Choose AI difficulty:

In [ ]:
print("Select your opponent:")
print("  1: Random (easiest)")
print("  2: Heuristic (medium - 65% vs Random)")
print("  3: Monte Carlo Fast (hard - 10 rollouts)")

opponent_choice = input("Opponent [1-3]: ").strip()

if opponent_choice == "1":
    ai_agent = RandomAgent("AI-Random")
elif opponent_choice == "2":
    ai_agent = HeuristicAgent("AI-Heuristic")
elif opponent_choice == "3":
    ai_agent = FastMonteCarloAgent("AI-MC-Fast")
else:
    print("Invalid choice, defaulting to Heuristic")
    ai_agent = HeuristicAgent("AI-Heuristic")

print(f"\n✓ Opponent: {ai_agent.name}")

## Select Your Side

In [ ]:
print("Select your player:")
print("  1: Player 1 (X) - moves first")
print("  2: Player 2 (O) - moves second")

player_choice = input("Player [1-2]: ").strip()

if player_choice == "1":
    human_player = Player.ONE
    ai_player = Player.TWO
elif player_choice == "2":
    human_player = Player.TWO
    ai_player = Player.ONE
else:
    print("Invalid choice, defaulting to Player 1")
    human_player = Player.ONE
    ai_player = Player.TWO

print(f"\n✓ You are Player {'1 (X)' if human_player == Player.ONE else '2 (O)'}")

## Start Game

Now run this cell to play! The game loop will prompt you for moves.

In [ ]:
# Create agents
human = HumanAgent("You")
agents = {
    human_player: human,
    ai_player: ai_agent
}

# Initialize engine
seed = 42
engine = GameEngine(seed=seed)

# Notify agents
agents[Player.ONE].game_start(Player.ONE, seed)
agents[Player.TWO].game_start(Player.TWO, seed)

print("=" * 60)
print(f"Starting game: {agents[Player.ONE].name} (X) vs {agents[Player.TWO].name} (O)")
print("Face-down cards (2,3,9,10) shown as ??")
print("=" * 60)
print()

# Game loop
while not engine.is_game_over():
    if engine.state.phase == Phase.DOGFIGHTS:
        # Start dogfight
        engine.begin_current_dogfight()
        
        # Show dogfight setup
        df = engine.current_dogfight
        if df:
            pos = df.position
            square = engine.state.get_square(pos[0], pos[1])
            if len(square.rocketmen) == 2:
                rm1, rm2 = square.rocketmen
                underdog_rm = rm1 if rm1.player == df.underdog else rm2
                other_rm = rm2 if rm1.player == df.underdog else rm1
                print(f"\n🥊 Dogfight at [{pos[0]},{pos[1]}]:")
                print(f"   {agents[df.underdog].name} (underdog, power {underdog_rm.power}) acts first")
                print(f"   vs {agents[df.other].name} (power {other_rm.power})")
        
        # Dogfight turns
        while not engine.is_dogfight_complete():
            current_player = engine.get_dogfight_current_actor()
            agent = agents[current_player]
            legal_actions = engine.get_dogfight_legal_actions_for_player(current_player)
            
            if agent == human:
                action_idx = agent.select_action(
                    engine.get_state_copy(),
                    legal_actions,
                    current_player
                )
            else:
                print(f"\n{agent.name} is thinking...")
                action_idx = agent.select_action(
                    engine.get_state_copy(),
                    legal_actions,
                    current_player
                )
                action = get_action_space().get_action(action_idx)
                print(f"{agent.name} plays: {action}")
            
            engine.apply_dogfight_turn_action(current_player, action_idx)
        
        # Finish dogfight
        result = engine.finish_current_dogfight()
        print("\n" + "-" * 60)
        print("📋 Dogfight complete!")
        print("-" * 60)
        
    else:
        # Placement phase
        current_player = engine.state.current_player
        agent = agents[current_player]
        legal_actions = engine.get_legal_actions()
        
        if agent == human:
            action_idx = agent.select_action(
                engine.get_state_copy(),
                legal_actions,
                current_player
            )
        else:
            print(f"\n{agent.name} is thinking...")
            action_idx = agent.select_action(
                engine.get_state_copy(),
                legal_actions,
                current_player
            )
            action = get_action_space().get_action(action_idx)
            print(f"{agent.name} plays: {action}")
        
        engine.apply_action(action_idx)

# Game over
print("\n" + "=" * 60)
print("GAME OVER")
print("=" * 60)

winner = engine.get_winner()
final_state = engine.get_state_copy()

# Display final board
print("\nFinal board:")
for row in range(3):
    row_str = []
    for col in range(3):
        square = final_state.get_square(row, col)
        if not square.rocketmen:
            row_str.append("   .   ")
        elif len(square.rocketmen) == 1:
            rm = square.rocketmen[0]
            owner = "X" if rm.player == Player.ONE else "O"
            row_str.append(f"   {owner}   ")
    print(f"  {row}: {'|'.join(row_str)}")

# Announce winner
print()
if winner is None:
    print("Result: DRAW")
elif winner == human_player:
    print("🎉 YOU WIN! Congratulations!")
else:
    print("AI wins. Better luck next time!")

# Display final score
p1_score = final_state.count_controlled_squares(Player.ONE)
p2_score = final_state.count_controlled_squares(Player.TWO)
print(f"\nFinal score: Player 1 (X): {p1_score}, Player 2 (O): {p2_score}")

# Notify agents
agents[Player.ONE].game_end(final_state, winner)
agents[Player.TWO].game_end(final_state, winner)

---

## Play Again?

To play another game, simply run the "Start Game" cell again!

Or [return to home](index.ipynb) to try other modes.